In [1]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [2]:
folder_path = 'UTKFace'

In [3]:
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
    age.append(int(file.split('_')[0]))
    gender.append(int(file.split('_')[1]))
    img_path.append(file)

In [4]:
len(age)

23708

In [5]:
df = pd.DataFrame({'age': age, 'gender':gender,'img':img_path})

In [6]:
df.shape

(23708, 3)

In [8]:
df.sample(10)

,age,gender,img
6365,26,0,26_0_2_20170116183004847.jpg.chip.jpg
20513,60,0,60_0_3_20170119210242095.jpg.chip.jpg
17162,47,0,47_0_0_20170117160014951.jpg.chip.jpg
16821,45,0,45_0_3_20170119203905073.jpg.chip.jpg
6283,26,0,26_0_2_20170104022540590.jpg.chip.jpg
8338,27,1,27_1_1_20170116000504802.jpg.chip.jpg
22916,85,1,85_1_0_20170110141027202.jpg.chip.jpg
1896,1,0,1_0_2_20161219203335428.jpg.chip.jpg
3538,22,1,22_1_1_20170114031355312.jpg.chip.jpg
18179,50,1,50_1_0_20170117121926916.jpg.chip.jpg


In [10]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                   )

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_dataframe(train_df,directory=folder_path,x_col='img',y_col=['age','gender'],target_size=(200,200),class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,directory=folder_path,x_col='img',y_col=['age','gender'],target_size=(200,200),class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [18]:
from keras.applications import VGG16
from keras.layers import *
from keras.models import Model

In [20]:
vggnet = VGG16(include_top=False,input_shape=(200,200,3))

In [27]:
vggnet.trainable=False
output = vggnet.layers[-1].output
flatten = Flatten()(output)


dense1 = Dense(512,activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [28]:
model = Model(inputs=vggnet.input, outputs=[output1,output2])

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 200, 200, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 200, 200, 64)         36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)         0         ['block1_conv2[0][0]']        
                                                                                            

In [30]:
model.compile(optimizer='adam',loss={'age':'mae','gender':'binary_crossentropy'},metrics={'age':'mae','gender':'accuracy'},loss_weights={'age':1,'gender':99})

In [ ]:
model.fit(train_generator,batch_size=128,epochs=1,validation_data=test_generator)

356/625 [================>.............] - ETA: 12:20 - loss: 65.3781 - age_loss: 11.3253 - gender_loss: 0.5460 - age_mae: 11.3253 - gender_accuracy: 0.7190